## Imports
All imports needed here

In [1]:
import os
import time
import pickle
import pandas as pd
import numpy as np
import fromGenoToMLData as fm

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, log_loss, roc_auc_score, confusion_matrix, cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

## GenModels_1SmE function
Function that obtains the level 1 expert of one repository in Meta Learning 1SmE

Parameters:

&emsp;__workPath__: String with your workpath<br>
&emsp;__handlerMLdata__: handler that contains the *.dataForML files with the data<br>
&emsp;__k__: Number of folds generated from the repo<br>

Return value:

&emsp;A dictionary with the algorithm name and the best model for this algorithm

In [2]:
def genModels_1SmE(workPath,
                   handlerMLdata,
                   k):
    ## Initialize variables
    
    # Here we chose the algorithms we will use (in future this will be a argument)
    algsML = [RandomForestClassifier(), 
              LogisticRegression(solver='lbfgs'), 
              DecisionTreeClassifier()]
    
    # These will be the columns for our final data table
    algs_cols = ["Fold", 
                 "Algorithm", 
                 "AUC_Percent", 
                 "Accuracy_Percent", 
                 "Balanced_Accuracy_Percent", 
                 "Log_Loss", "Sensitivity", 
                 "Specificity", 
                 "PPV", 
                 "NPV", 
                 "Runtime_Seconds"]

    algs_table = pd.DataFrame(columns=algs_cols)

    # These will be the columns for our total data table (same columns as before)
    folds_cols =  algs_cols
    folds_table = pd.DataFrame(columns=folds_cols)

    models = {}      # A dictionary with all models
    finalModels = {} # A dictionary with final models

    for i in range(1, k + 1):
        models[i] = {}
    
    # Prepare data for training
    X = pd.read_csv(handlerMLdata["train1mldata"], header=0, delim_whitespace=True) # Read data
    X = X.loc[X["AGE"] != -9] # Delete data with invalid age
    X.dropna(inplace=True) # Delete all NaN values
    X.drop("ID", 1, inplace=True) # We dont need ID for the training


    # Get the target variable
    Y = X["PHENO"]
    X.drop("PHENO", 1, inplace=True)

    # Process it
    Y.columns = ["PHENO"]
    Y = Y.to_numpy()
    le = LabelEncoder()
    le.fit(Y)
    Y_le = le.transform(Y)

    X_np = X.to_numpy() # We need our data in numpy format for training (it is in pandas originally)

    # Make folds for data training
    np.random.seed(1234)
    myfolds = KFold(n_splits=k)

    for alg in algsML: # For each algorithm

        # Get algorithm name and print it
        name = alg.__class__.__name__ 

        print()
        print("#" * 30)
        print(name + "\n")

        fold = 1

        for train_index, test_index in myfolds.split(X_np): # For each fold

            print()
            print("\t" + "#" * 26)
            print("\t" + "Fold " + str(fold) + "\n")
            
            start_time = time.time()

            # Get the train and test sets
            X_train, X_test = X_np[train_index], X_np[test_index]
            Y_train, Y_test = Y_le[train_index], Y_le[test_index]
            
            # Train
            np.random.seed(1234)
            alg.fit(X_train, Y_train)

            # Save the model
            models[fold][name] = alg

            test_predictions = alg.predict_proba(X_test)
            rocauc = roc_auc_score(Y_test, test_predictions[:, 1])
            print("\t\tAUC: {:.4%}".format(rocauc))

            test_predictions = alg.predict(X_test)
            acc = accuracy_score(Y_test, test_predictions)
            print("\t\tAccuracy: {:.4%}".format(acc))

            test_predictions = alg.predict(X_test)
            balacc = balanced_accuracy_score(Y_test, test_predictions)
            print("\t\tBalanced Accuracy: {:.4%}".format(balacc))

            test_predictions = alg.predict(X_test)
            kappa = cohen_kappa_score(Y_test, test_predictions)
            print("\t\tKappa: {:.4%}".format(kappa))

            CM = confusion_matrix(Y_test, test_predictions)
            TN = CM[0][0]
            FN = CM[1][0]
            TP = CM[1][1]
            FP = CM[0][1]
            sensitivity = TP / (TP + FN)
            specificity = TN / (TN + FP)
            PPV = TP / (TP + FP)
            NPV = TN / (TN + FN)

            test_predictions = alg.predict_proba(X_test)
            ll = log_loss(Y_test, test_predictions)
            print("\t\tLog Loss: {:.4}".format(ll))

            end_time = time.time()
            elapsed_time = (end_time - start_time)
            print("\t\tRuntime in seconds: {:.4}".format(elapsed_time))

            # Add the entry to the table
            folds_entry = pd.DataFrame([[fold, name, rocauc * 100, acc * 100, balacc * 100, ll, sensitivity, specificity, PPV, NPV, elapsed_time]], columns=folds_cols)
            folds_table = folds_table.append(folds_entry)

            fold = fold + 1

    folds_table.index = range(len(folds_table.index)) # Give to the index the correct names

    # Get the best model for every Algorithm and add it to the final table
    for alg in algsML:
        name = alg.__class__.__name__
        alg_best_entry = folds_table.iloc[folds_table[folds_table["Algorithm"] == name]['Balanced_Accuracy_Percent'].idxmax()]
        finalModels[name] = models[alg_best_entry["Fold"]][name]
        algs_table = algs_table.append(alg_best_entry)
    
    algs_table.index = range(len(algs_table.index))
    print("\n\n")
    print(algs_table)
    print("\n")


    # Save models in file
    with open(workPath + "models.pydat", 'wb') as finalModels_file:
        pickle.dump(finalModels, finalModels_file)

    # We save the columns used in training for later
    finalModels["columns"] = X.columns

    return finalModels

## TrainAndTestMML_1SmE function
Function that creates the data for MetaML and obtains the level 2 expert of one repository in Meta Learning 1SmE

Parameters:

&emsp;__experts_L1__: Experts obtained from the function genModels<br>
&emsp;__handlerMLdata__: Handler that contains the *.dataForML files with the data<br>
&emsp;__workPath__: String with your workpath<br>

Return value:

&emsp;The model generated by training the metaset

In [3]:
def trainAndTestMML_1SmE(experts_L1,
                         handlerMLdata,
                         workPath="/home/edusal/MetaLearning/metaML/Modelos/"):

    # Initialize variables
    
    # These are the algorithms we use in Meta-ML
    algsMML = [RandomForestClassifier(), LogisticRegression(solver='lbfgs'), DecisionTreeClassifier()]

    # These will be the columns for our final data table
    algs_cols = ["Fold", 
                 "Algorithm", 
                 "AUC_Percent", 
                 "Accuracy_Percent", 
                 "Balanced_Accuracy_Percent", 
                 "Log_Loss", 
                 "Sensitivity", 
                 "Specificity", 
                 "PPV", 
                 "NPV", 
                 "Runtime_Seconds"]
    algs_table = pd.DataFrame(columns=algs_cols)

    # These will be the columns for our total data table (same columns as before)
    folds_cols = algs_cols
    folds_table = pd.DataFrame(columns=folds_cols)
    
    models = {}      # A dictionary with all models
    finalModels = {} # A dictionary with final models
    k = 5            # Number of folds

    for i in range(1, k + 1):
        models[i] = {}
    
    
    # Prepare data for training
    dataTest = pd.read_csv(handlerMLdata["test1mldata"], header=0, delim_whitespace=True) # Read data
    totalData = dataTest

    totalData = totalData.loc[totalData["AGE"] != -9] # Remove data with invalid age
    ID = totalData["ID"]
    totalData.drop("ID", 1, inplace=True) # We dont need id for training
    
    # Get the target variable
    Y = totalData["PHENO"]
    totalData.drop("PHENO", 1, inplace=True)

    # Initialize metaSet with the target variable and ID
    Y.index = range(len(Y.index))
    ID.index = range(len(ID.index))

    metaSet = pd.concat([ID, Y], axis=1)

    # As an argument in future version
    algsML = [RandomForestClassifier(), LogisticRegression(solver='lbfgs'), DecisionTreeClassifier()]

    # Obtain the predictions for each Algorithm so we use them in the meta training
    for alg in algsML:
        name = alg.__class__.__name__
        preds = experts_L1[name].predict(totalData)
        metaSet["Pred-" + name] = preds

    # We assume MGSA type (we add age, sex and geno)
    cols = totalData.columns
    cols = [col for col in cols if not col.startswith("PC")]
    all_cols = totalData[cols]

    metaSet.index = range(len(metaSet.index))
    all_cols.index = range(len(all_cols.index))

    metaSet = pd.concat([metaSet, all_cols], axis=1)

    # We no longer need ID and Pheno in metaSet
    ID = metaSet["ID"]
    metaSet.drop("ID", 1, inplace=True)
    metaSet.drop("PHENO", 1, inplace=True)

    # Separate the new dataframe into train and test (75% train - 25% test)
    X_trainMML, X_testMML, Y_trainMML, Y_testMML = train_test_split(metaSet, Y, test_size=0.25)
    X_np = X_trainMML.to_numpy() # We need our data in numpy format

    # Process the target value
    Y_np = Y_trainMML.to_numpy()
    le = LabelEncoder()
    le.fit(Y_np)
    Y_le = le.transform(Y_np)
    
    # Make folds
    np.random.seed(1234)
    myfolds = KFold(n_splits=k)

    for alg in algsMML: # For each algorithm

        # Get the algorithm name and print it
        name = alg.__class__.__name__

        print()
        print("#" * 30)
        print(name + "\n")

        fold = 1

        for train_index, test_index in myfolds.split(X_np): # For each fold

            print()
            print("\t" + "#" * 26)
            print("\t" + "Fold " + str(fold) + "\n")
            
            start_time = time.time()
            np.random.seed(1234)
            
            # Get the train and test sets
            X_train, X_test = X_np[train_index], X_np[test_index]
            Y_train, Y_test = Y_le[train_index], Y_le[test_index]

            # Train
            alg.fit(X_train, Y_train)

            # Save the model
            models[fold][name] = alg

            test_predictions = alg.predict_proba(X_test)
            rocauc = roc_auc_score(Y_test, test_predictions[:, 1])
            print("\t\tAUC: {:.4%}".format(rocauc))

            test_predictions = alg.predict(X_test)
            acc = accuracy_score(Y_test, test_predictions)
            print("\t\tAccuracy: {:.4%}".format(acc))

            test_predictions = alg.predict(X_test)
            balacc = balanced_accuracy_score(Y_test, test_predictions)
            print("\t\tBalanced Accuracy: {:.4%}".format(balacc))

            test_predictions = alg.predict(X_test)
            kappa = cohen_kappa_score(Y_test, test_predictions)
            print("\t\tKappa: {:.4%}".format(kappa))

            CM = confusion_matrix(Y_test, test_predictions)
            TN = CM[0][0]
            FN = CM[1][0]
            TP = CM[1][1]
            FP = CM[0][1]
            sensitivity = TP / (TP + FN)
            specificity = TN / (TN + FP)
            PPV = TP / (TP + FP)
            NPV = TN / (TN + FN)

            test_predictions = alg.predict_proba(X_test)
            ll = log_loss(Y_test, test_predictions)
            print("\t\tLog Loss: {:.4}".format(ll))

            end_time = time.time()
            elapsed_time = (end_time - start_time)
            print("\t\tRuntime in seconds: {:.4}".format(elapsed_time))

            # Add the entry to the table
            folds_entry = pd.DataFrame([[fold, name, rocauc * 100, acc * 100, balacc * 100, ll, sensitivity, specificity, PPV, NPV, elapsed_time]], columns=algs_cols)
            folds_table = folds_table.append(folds_entry)

            fold = fold + 1

    # Save models
    with open(workPath + "models_MML.pydat", 'wb') as finalModels_file:
        pickle.dump(finalModels, finalModels_file)

    folds_table.index = range(len(folds_table.index))

    # Get the best model for every Algorithm and add it to the final table
    for alg in algsMML:
        name = alg.__class__.__name__
        alg_best_entry = folds_table.iloc[folds_table[folds_table["Algorithm"] == name]['Balanced_Accuracy_Percent'].idxmax()]
        finalModels[name] = models[alg_best_entry["Fold"]][name]
        algs_table = algs_table.append(alg_best_entry)

    algs_table.index = range(len(algs_table.index))
    print("\n\n")
    print(algs_table)
    print("\n")

    # Get the best model
    bestAlg = algs_table.iloc[algs_table['Balanced_Accuracy_Percent'].idxmax()]['Algorithm']

    # Create the return variable with the best model and the columns used in metatraining
    expert_L2 = {}
    expert_L2["model"] = finalModels[bestAlg]
    expert_L2["columns"] = metaSet.columns
    metaSet["ID"] = ID # Add ID to the metaSet
    
    # Save dataframes and results
    with open(workPath + "metaSet.pydat", 'wb') as metaSet_file:
        pickle.dump(metaSet, metaSet_file)

    with open(workPath + "expert_L2.pydat", 'wb') as expert_L2_file:
        pickle.dump(expert_L2, expert_L2_file)

    with open(workPath + "metaResults.pydat", 'wb') as metaResults_file:
        pickle.dump(folds_table, metaResults_file)

    return expert_L2

## PrepareFinalTest function
Function that generates the ML data for the repo reserved for the test part

Parameters:

&emsp;__workPath__: String with your workpath<br>
&emsp;__path2Geno__: Path to your genotype data<br>
&emsp;__path2Covs__:Path to your covariates data<br>
&emsp;__predictor__: What you want to predict<br>
&emsp;__addit__: <br>
&emsp;__snpsToPull__: SNPs selected from the spanish repo, to use them to extract variables in the testing repo<br>
&emsp;__path2plink__: Path to plink<br>

Return value:

&emsp;Data prepare for the training

In [ ]:
def prepareFinalTest(workPath,
                     path2Geno,
                     path2Covs,
                     predictor,
                     snpsToPull,
                     path2plink="/home/edusal/packages/",
                     addit="NA"):

    command = "cp " + path2Covs + ".cov " + workPath
    print("The command to run: " + command)
    os.system(command)

    # Get the repo's handler
    handler = fm.getHandlerToGenotypeData(geno=path2Geno,
                                          covs=path2Covs,
                                          id="IID",
                                          fid="FID",
                                          predictor=predictor,
                                          pheno=workPath + "/MyPhenotype")

    geno = os.path.basename(handler["geno"])
    pheno = os.path.basename(handler["pheno"])
    cov = os.path.basename(handler["covs"])
    path2Genotype = os.path.dirname(handler["geno"]) + "/"
    prefix = "g-" + geno + "-p-" + pheno + "-c-" + cov + "-a-" + addit
    fprefix = workPath + "/" + prefix
    
    # Set the SNPs to pull to the ones selected in the spanish set
    handler["snpsToPull"] = snpsToPull  

    # Run Plink
    command = path2plink + "plink --bfile " + path2Genotype + geno + " --extract " +\
        handler["snpsToPull"] + " --recode A --out " + fprefix + ".reduced_genos"
    print("Running command " + command + "\n")
    os.system(command)

    # Exports SNP list for extraction in validation set
    command = "cut -f 1 " + handler["snpsToPull"] + " > " + fprefix + ".reduced_genos_snpList"
    print("Running command " + command + "\n")
    os.system(command)

    handler["rgenosSnpList"] = fprefix + ".reduced_genos_snpList"

    # Generate the .dataForML file
    mldatahandler = fm.fromSNPs2MLdata(handler=handler,
                                       addit="NA",
                                       path2plink=path2plink,
                                       predictor=predictor)

    # Save it
    with open(workPath + "mldatahandler.pydat", 'wb') as mldatahandler_file:
        pickle.dump(mldatahandler, mldatahandler_file)

    return mldatahandler


## FinalTest_1SmE function
Function that obtains the prediction in the test repository in the Meta Learning 1SmE.

Parameters:

&emsp;__workPath__: String with your workpath<br>
&emsp;__expert_L2__: Expert returned by the MML 1SmE<br>
&emsp;__handlerTest__: Handler with the mldata of the test set<br>
&emsp;__experts_L1__: Experts used in the generation of the metaset<br>

Return value:

&emsp;Results of training with with test set.

In [ ]:
def finalTest_1SmE(workPath,
                   expert_L2,
                   handlerTest,
                   experts_L1):

    dataFinal = pd.read_csv(handlerTest["train1mldata"], header=0, delim_whitespace=True) # Read data

    # same preprocess applied to the training
    ID = dataFinal["ID"]
    dataFinal.drop("ID", 1, inplace=True)

    Y = dataFinal["PHENO"]
    dataFinal.drop("PHENO", 1, inplace=True)

    Y_np = Y.to_numpy()
    le = LabelEncoder()
    le.fit(Y_np)
    Y_le = le.transform(Y_np)

    Y.index = range(len(Y.index))
    ID.index = range(len(ID.index))

    # dataframe which will contain the predictions from each model
    metaSet = pd.concat([ID, Y], axis=1)

    aux = dataFinal

    # Add variables set to zero, present in each expert but not in the test data
    diff1 = set(experts_L1["columns"]) - set(aux.columns)
    if len(diff1) > 0:
        new_cols = pd.DataFrame(0, index=np.arange(len(aux)), columns=diff1)

        aux.index = range(len(aux.index))
        aux = pd.concat([aux, new_cols], axis=1)
    
    # Remove variables in the test data and not in the data used for training each expert
    diff2 = set(aux.columns) - set(experts_L1["columns"])
    if len(diff2) > 0:
        aux.drop(diff2, 1, inplace=True)

    # As argument in future version
    algsML = [RandomForestClassifier(), LogisticRegression(solver='lbfgs'), DecisionTreeClassifier()]
    
    # Obtain the predictions for each Algorithm so we use them in the meta training
    for alg in algsML:
        name = alg.__class__.__name__

        preds = experts_L1[name].predict(aux)
        metaSet["Pred-" + name] = preds

    # We assume its type MGSA and add age, sex and geno
    cols = dataFinal.columns
    cols = [col for col in cols if not col.startswith("PC")]
    all_cols = dataFinal[cols]

    metaSet.index = range(len(metaSet.index))
    all_cols.index = range(len(all_cols.index))

    metaSet = pd.concat([metaSet, all_cols], axis=1)

    ID = metaSet["ID"]
    metaSet.drop("ID", 1, inplace=True)
    metaSet.drop("PHENO", 1, inplace=True)

    aux = metaSet
    
    # add variables set to zero present in the MML model but not in the data
    diff1 = set(expert_L2["columns"]) - set(aux.columns)
    if len(diff1) > 0:
        new_cols = pd.DataFrame(0, index=np.arange(len(aux)), columns=diff1)

        aux.index = range(len(aux.index))
        aux = pd.concat([aux, new_cols], axis=1)

    # remove variables in the metaset and not in the data used for training the MML model
    diff2 = set(aux.columns) - set(expert_L2["columns"])
    if len(diff2) > 0:
        aux.drop(diff2, 1, inplace=True)

    # Put the columns in the right order
    metaSet = aux[expert_L2["columns"].tolist()]

    # Save metaSet
    with open(workPath + "metaSet.pydat", 'wb') as metaSet_file:
        pickle.dump(metaSet, metaSet_file)

    # Get final results
    test_predictions = expert_L2["model"].predict_proba(metaSet)
    rocauc = roc_auc_score(Y_le, test_predictions[:, 1])
    print("\t\tAUC: {:.4%}".format(rocauc))

    test_predictions = expert_L2["model"].predict(metaSet)
    acc = accuracy_score(Y_le, test_predictions)
    print("\t\tAccuracy: {:.4%}".format(acc))

    test_predictions = expert_L2["model"].predict(metaSet)
    balacc = balanced_accuracy_score(Y_le, test_predictions)
    print("\t\tBalanced Accuracy: {:.4%}".format(balacc))

    test_predictions = expert_L2["model"].predict(metaSet)
    kappa = cohen_kappa_score(Y_le, test_predictions)
    print("\t\tKappa: {:.4%}".format(kappa))

    CM = confusion_matrix(Y_le, test_predictions)

    test_predictions = expert_L2["model"].predict_proba(metaSet)
    ll = log_loss(Y_le, test_predictions)
    print("\t\tLog Loss: {:.4}".format(ll))

    # Save final results
    finalResults = {}
    finalResults["PHENO"] = Y
    finalResults["AUC"] = rocauc
    finalResults["Accuracy"] = acc
    finalResults["Balanced Accuracy"] = balacc
    finalResults["Kappa"] = kappa

    finalResults["confMat"] = CM

    return(finalResults)
